In [1]:

# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [2]:
# import sys
# sys.path.append('/content/tesis-main/')

from dotenv import load_dotenv

load_dotenv()

True

In [3]:

# import drive as drive_downloader
# drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [8]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from models.resnet18 import get_model as get_resnet18
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from clustering.model import clustering_function
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset_loader = load_dataset(VALIDATION_DATASET.keys())

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: piodanilod (hortensias). Use `wandb login --relogin` to force relogin


True

In [9]:
from properties import MODELS_ENUM, PREPROCESSORS, ALGORITHM, GROUPER_FUNCTIONS

DEFAULT_PARAMS = {
      "dataset": dataset_loader,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }


EXPERIMENT_DEFINITIONS= [
    {
        'name': 'ResnetAglomerative',
        'model': "RESNET18",
        'algorithm': "AGGLOMERATIVE",
        'params': {"n_clusters": None, "distance_threshold": 50},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    },
    {
        'name': 'SimilarityClustering',
        'model': "RESNET18",
        'algorithm': "SIMILARITY",
        'params': {"threshold": 0.999},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    }
]

for exp_definition in EXPERIMENT_DEFINITIONS:
  params = DEFAULT_PARAMS.copy()
  params["model"] = MODELS_ENUM[exp_definition["model"]](device)
  params["preprocessing"] = PREPROCESSORS[exp_definition["preprocessing"]]
  algorithm = ALGORITHM[exp_definition["algorithm"]](**exp_definition["params"])
  grouper_function = GROUPER_FUNCTIONS[exp_definition["grouper_function"]](algorithm)
  params["grouper_function"] = grouper_function
  params["name"] = exp_definition["name"]
  params["dataset"] = dataset_loader()

  run = wandb.init(project="ort-tesis",group=exp_definition['name'],config=exp_definition, entity="hortensias")
  result = experiment(**params)
  print(result._asdict())
  run.log({
    "Precision": result.precision.mean,
    "Accuracy": result.accuracy.mean,
    "F1": result.f1.mean,
    "Recall": result.recall.mean,
    "Precision_STD": result.precision.std,
    "Accuracy_STD": result.accuracy.std,
    "F1_STD": result.f1.std,
    "Recall_STD": result.recall.std,
  })
  run.finish()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/pio/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 47.0MB/s]



Experiment name: ResnetAglomerative Airbnb.mp4
Precision - Mean: 0.61, std: 0.0426408229
Recall - Mean: 0.50, std: 0.0542994165
F1-Score - Mean: 0.51, std: 0.0533845729
Accuracy - Mean: 0.50, std: 0.0542994165

Experiment name: ResnetAglomerative AVG
Precision - Mean: 0.61, std: 0.0000000000
Recall - Mean: 0.50, std: 0.0000000000
F1-Score - Mean: 0.51, std: 0.0000000000
Accuracy - Mean: 0.50, std: 0.0000000000
{'precision': MetricsResults(mean=0.6109762163769819, std=0.0), 'recall': MetricsResults(mean=0.5017724413950829, std=0.0), 'f1': MetricsResults(mean=0.5101993366100538, std=0.0), 'accuracy': MetricsResults(mean=0.5017724413950829, std=0.0)}


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.50177
Accuracy_STD,0.0
F1,0.5102



Experiment name: SimilarityClustering Airbnb.mp4
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000

Experiment name: SimilarityClustering AVG
Precision - Mean: 0.94, std: 0.0000000000
Recall - Mean: 0.49, std: 0.0000000000
F1-Score - Mean: 0.61, std: 0.0000000000
Accuracy - Mean: 0.49, std: 0.0000000000
{'precision': MetricsResults(mean=0.9399112694007534, std=0.0), 'recall': MetricsResults(mean=0.4905660377358491, std=0.0), 'f1': MetricsResults(mean=0.6145718925642055, std=0.0), 'accuracy': MetricsResults(mean=0.4905660377358491, std=0.0)}


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.49057
Accuracy_STD,0.0
F1,0.61457


In [12]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')


AttributeError: 'YOLO' object has no attribute 'summary'

In [13]:
result = model.val()
result

Ultralytics YOLOv8.1.3 🚀 Python-3.11.3 torch-2.1.0 CPU (Apple M1)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco.yaml' images not found ⚠️, missing path '/Users/pio/postgrado/tesis/datasets/coco/val2017.txt'


100%|██████████| 169M/169M [00:06<00:00, 28.0MB/s] 
Unzipping /Users/pio/postgrado/tesis/datasets/coco2017labels-segments.zip to /Users/pio/postgrado/tesis/datasets/coco...: 100%|██████████| 122232/122232 [00:10<00:00, 11401.04file/s]


KeyboardInterrupt: 

In [19]:
rwsult = model('/Users/pio/postgrado/tesis/tesis/data/output.mp4')
rwsult[0].boxes



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/372) /Users/pio/postgrado/tesis/tesis/data/output.mp4: 640x320 2 cell phones, 59.3ms
video 1/1 (2/372) /Users/pio/postgrado/tesis/tesis/data/output.mp4: 640x320 1 cell phone, 50.3ms
video 1/1 (3/372) /Users/pio/postgrado/tesis/tesis/data/output.mp4: 640x320 1 cell phone, 41.6ms
video 1/1 (4/372) /Users/pio/postgrado/tesis/tesis/data/output.mp4: 640x320 1 tv, 39.9ms
video 1/1 (5/372) /Users/pio/postgrado/tesis/tesis/data/output.mp4: 640x320

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([67., 67.])
conf: tensor([0.3059, 0.2533])
data: tensor([[1.6263e+00, 4.4522e+02, 1.0542e+02, 5.5189e+02, 3.0590e-01, 6.7000e+01],
        [1.6810e+00, 3.1726e+02, 1.0339e+02, 4.3934e+02, 2.5326e-01, 6.7000e+01]])
id: None
is_track: False
orig_shape: (720, 344)
shape: torch.Size([2, 6])
xywh: tensor([[ 53.5219, 498.5569, 103.7914, 106.6740],
        [ 52.5339, 378.3033, 101.7057, 122.0791]])
xywhn: tensor([[0.1556, 0.6924, 0.3017, 0.1482],
        [0.1527, 0.5254, 0.2957, 0.1696]])
xyxy: tensor([[  1.6263, 445.2199, 105.4176, 551.8939],
        [  1.6810, 317.2637, 103.3867, 439.3428]])
xyxyn: tensor([[0.0047, 0.6184, 0.3064, 0.7665],
        [0.0049, 0.4406, 0.3005, 0.6102]])